In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['GAS']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('group a streptococcus', 431),
 ('group a streptococci', 84),
 ('gamma activated sequence', 59),
 ('gamma activated site', 38),
 ('group a streptococcal', 36),
 ('gastrodin', 33),
 ('gastrocnemius', 21),
 ('gastric acid secretion', 17),
 ('group a streptococcus pyogenes', 16),
 ('gastrin', 15),
 ('gamma interferon activated site', 13),
 ('gamma interferon activated sequence', 8),
 ('gastric acid suppressants', 5),
 ('goal attainment scale', 5),
 ('glasgow aneurysm score', 4),
 ('group astreptococcus', 4),
 ('group a beta hemolytic streptococcus', 4),
 ('group a β hemolytic streptococcus', 4),
 ('gingerol aspirinate', 3),
 ('gaming addiction scale', 3),
 ('gas mixture', 3)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-02 03:28:19] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [6]:
result = [grounding_map, names, pos_labels]

In [5]:
result

NameError: name 'result' is not defined

In [6]:
grounding_map, names, pos_labels = [{'gaming addiction scale': 'ungrounded',
  'gamma activated sequence': 'gamma_activated_sequence',
  'gamma activated site': 'gamma_activated_sequence',
  'gamma interferon activated sequence': 'gamma_activated_sequence',
  'gamma interferon activated site': 'gamma_activated_sequence',
  'gas mixture': 'ungrounded',
  'gastric acid secretion': 'GO:GO:0001696',
  'gastric acid suppressants': 'ungrounded',
  'gastrin': 'HGNC:4164',
  'gastrocnemius': 'NCIT:C32666',
  'gastrodin': 'CHEBI:CHEBI:80828',
  'gingerol aspirinate': 'ungrounded',
  'glasgow aneurysm score': 'ungrounded',
  'goal attainment scale': 'ungrounded',
  'group a beta hemolytic streptococcus': 'TAXONOMY:36470',
  'group a streptococcal': 'TAXONOMY:36470',
  'group a streptococci': 'TAXONOMY:36470',
  'group a streptococcus': 'TAXONOMY:36470',
  'group a streptococcus pyogenes': 'TAXONOMY:36470',
  'group a β hemolytic streptococcus': 'TAXONOMY:36470',
  'group astreptococcus': 'TAXONOMY:36470'},
 {'gamma_activated_sequence': 'gamma_activated_sequence',
  'GO:GO:0001696': 'gastric acid secretion',
  'HGNC:4164': 'GAST',
  'NCIT:C32666': 'Gastrocnemius Muscle',
  'CHEBI:CHEBI:80828': 'Gastrodin',
  'TAXONOMY:36470': "Streptococcus sp. 'group A'"},
 ['CHEBI:CHEBI:80828', 'HGNC:4164', 'TAXONOMY:36470', 'NCIT:C32666']]

In [7]:
excluded_longforms = []

In [8]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [9]:
additional_entities = {'HGNC:4164': ['GAST', ['GAS']]}

In [10]:
unambiguous_agent_texts = {}

In [11]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:4164', 'HGNC:4164', 121)]

In [15]:
intersection2

[('GO:GO:0001696', 'HGNC:4164', 0),
 ('ungrounded', 'HGNC:4164', 0),
 ('TAXONOMY:36470', 'HGNC:4164', 0),
 ('gamma_activated_sequence', 'HGNC:4164', 0),
 ('HGNC:4164', 'HGNC:4164', 1),
 ('CHEBI:CHEBI:80828', 'HGNC:4164', 0),
 ('NCIT:C32666', 'HGNC:4164', 0)]

In [16]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [17]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [18]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-30 01:21:58] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-30 01:22:10] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9432987562942685 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [19]:
classifier.stats

{'label_distribution': {'GO:GO:0001696': 17,
  'ungrounded': 13,
  'TAXONOMY:36470': 422,
  'gamma_activated_sequence': 105,
  'HGNC:4164': 22,
  'CHEBI:CHEBI:80828': 29,
  'NCIT:C32666': 15},
 'f1': {'mean': 0.943299, 'std': 0.030306},
 'precision': {'mean': 0.935696, 'std': 0.026084},
 'recall': {'mean': 0.960972, 'std': 0.028641},
 'HGNC:4164': {'f1': {'mean': 0.847619, 'std': 0.189042},
  'pr': {'mean': 0.8, 'std': 0.252982},
  'rc': {'mean': 0.95, 'std': 0.1}},
 'CHEBI:CHEBI:80828': {'f1': {'mean': 0.808225, 'std': 0.148464},
  'pr': {'mean': 0.746667, 'std': 0.202868},
  'rc': {'mean': 0.926667, 'std': 0.090431}},
 'GO:GO:0001696': {'f1': {'mean': 0.595238, 'std': 0.063888},
  'pr': {'mean': 0.533333, 'std': 0.124722},
  'rc': {'mean': 0.8, 'std': 0.244949}},
 'ungrounded': {'f1': {'mean': 0.566667, 'std': 0.08165},
  'pr': {'mean': 0.4, 'std': 0.08165},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'NCIT:C32666': {'f1': {'mean': 0.831429, 'std': 0.183481},
  'pr': {'mean': 0.8, 'std': 0.

In [20]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [21]:
disamb.dump(model_name, results_path)

In [22]:
print(disamb.info())

Disambiguation model for GAS

Produces the disambiguations:
	GAST*	HGNC:4164
	Gastrocnemius Muscle*	NCIT:C32666
	Gastrodin*	CHEBI:CHEBI:80828
	Streptococcus sp. 'group A'*	TAXONOMY:36470
	gamma_activated_sequence	gamma_activated_sequence
	gastric acid secretion	GO:GO:0001696

Class level metrics:
--------------------
Grounding                  	Count	F1     
Streptococcus sp. 'group A'*	422	0.96211
   gamma_activated_sequence	105	0.95167
                  Gastrodin*	 29	0.80822
                       GAST*	 22	0.84762
     gastric acid secretion	 17	0.59524
       Gastrocnemius Muscle*	 15	0.83143
                 Ungrounded	 13	0.56667

Weighted Metrics:
-----------------
	F1 score:	0.9433
	Precision:	0.9357
	Recall:		0.96097

* Positive labels
See Docstring for explanation



In [23]:
model_to_s3(disamb)